In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

torch.manual_seed(1228)

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

batch_size = 1

var_list = [300]

green_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\green'
yellow_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\yellow'
red_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\red'
blue_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\blue'

In [13]:
green_set = datasets.ImageFolder(
    green_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

yellow_set = datasets.ImageFolder(
    yellow_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

red_set = datasets.ImageFolder(
    red_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

blue_set = datasets.ImageFolder(
    blue_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)


green_train, green_test = torch.utils.data.random_split(green_set, [250, 50])  
yellow_train, yellow_test = torch.utils.data.random_split(yellow_set, [250, 50])  
red_train, red_test = torch.utils.data.random_split(red_set, [250, 50]) 
blue_train, blue_test = torch.utils.data.random_split(blue_set, [250, 50]) 


normal_set = green_set + blue_set
#abnormal_set = red_set + yellow_set
abnormal_set = red_set

train_dataset1 = list(green_train)[:int(len(green_train)*0.2)] + list(blue_train)[:int(len(blue_train)*0.8)]
train_dataset2 = list(green_train)[:int(len(green_train)*0.2)] + list(blue_train)[:int(len(blue_train)*0.8)]
test_dataset = green_test + blue_test

In [14]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))

        # Decoder        
        self.tran_cnn_layer1 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2, padding=0),
            nn.ReLU())

        self.tran_cnn_layer2 = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size = 2, stride = 2, padding=0),
            nn.Sigmoid())
            
            
    def forward(self, x):
        output = self.cnn_layer1(x)
        output = self.cnn_layer2(output)      
        output = self.tran_cnn_layer1(output)
        output = self.tran_cnn_layer2(output)
        
        return output

In [15]:
import math
    
def myEquation4(x, model):    
    normal_err = 0
    abnormal_err = 0    
    
    for i, (X_normal, _) in enumerate(normal_loader):
        X_normal = X_normal.cuda()
        output = model(X_normal)
        loss = criterion(X_normal, output)
        
        if loss.item() > x:
            normal_err += 1
        
    for i, (X_abnormal, _) in enumerate(abnormal_loader):
        X_abnormal = X_abnormal.cuda()
        output = model(X_abnormal)
        loss = criterion(X_abnormal, output)
        
        if loss.item() < x:
            abnormal_err += 1
      
    err = ((normal_err/len(normal_loader)) + (abnormal_err/len(abnormal_loader)))/2.0
    
    return err

gr = (math.sqrt(5) + 1) / 2

def gss(f, a, b,  model, tol=1e-8):
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c, model) < f(d, model):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) / gr
        d = a + (b - a) / gr

    return (b + a) / 2

In [16]:
normal_loader = torch.utils.data.DataLoader(normal_set, batch_size=batch_size, shuffle=True)
abnormal_loader = torch.utils.data.DataLoader(abnormal_set, batch_size=batch_size, shuffle=True)

criterion = nn.MSELoss()

for i, r in enumerate(var_list, 0):
    model = ConvAutoEncoder().to(device)
    model = torch.load(f'model_per_round/round_{r}_weight.pth')

    threshold = gss(myEquation4, 0.0, 0.01, model)
    err = myEquation4(threshold, model)
    print(f"Number of round : {r}\nthreshold : {threshold}\nerr : {err}\n")

    with open(f'data/threshold by gss.txt', 'a') as f:
        f.write(f'{threshold}\n')
        
    

Number of round : 300
threshold : 0.004390575316659413
err : 0.0

